## Glosario
---

***status***:
- "Default" = 0
- "Fully Paid" = 1 
- "Charged Off" = 2

***term***:
- "36 months" = 0
- "60 months" = 1

***home_ownership***:
- "MORTGAGE" = 0
- "RENT" = 1
- "OTHER" = 2
- "OWN" = 3

***outcome***:
- "default" = 0
- "paid off" = 1

***emp_len_***:
- " < 1 Year" = 0
- " > 1 Year" = 1

In [1]:
import pandas as pd
import numpy as np
from sklearn import preprocessing
from sklearn.neighbors import KNeighborsClassifier
from sklearn import metrics
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

In [2]:
df = pd.read_csv("./loan_data_modified_np.csv")
df.head()  # muestra 5 por defecto

,index_,status,loan_amnt,term,annual_inc,dti,payment_inc_ratio,revol_bal,revol_util,purpose,...,delinq_2yrs_zero,pub_rec_zero,open_acc,grade,outcome,emp_length,purpose_,home_,emp_len_,borrower_score
0,23063,1,9000,0,50000,17.11,7.08600,9093,74.5,debt_consolidation,...,1,1,6,5.4,1,6,debt_consolidation,OWN,1,0.45
1,23065,1,2500,0,33600,3.39,2.73250,3375,7.8,medical,...,1,1,15,6.2,1,11,medical,OWN,1,0.60
2,23067,1,6000,0,24000,5.75,10.54150,2327,43.1,wedding,...,1,1,6,5.0,1,2,other,OWN,1,0.55
3,23077,1,7000,0,24000,14.85,10.82900,268,0.7,major_purchase,...,1,1,7,6.6,1,1,major_purchase,OWN,1,0.50
4,3774,2,8000,0,48000,22.25,6.99025,8784,73.2,debt_consolidation,...,0,0,14,3.6,0,4,debt_consolidation,RENT,1,0.45
